In [2]:
import numpy as np
import pandas as pd

In [3]:
# batch_x , batch_x_mark , natch_y_mark

In [4]:
df = pd.read_csv("data/ACBI.csv")
size = [96,48,24]

seq_len = size[0]
label_len = size[1]
pred_len = size[2]

In [5]:
complete_df = df.iloc[-seq_len:]
complete_df.head()

,date,open,high,low,close,volume
57407,2021-12-30 19:47:00+00:00,29.070,29.070,29.060,29.060,530.0
57408,2021-12-30 19:52:00+00:00,29.040,29.040,29.040,29.040,238.0
57409,2021-12-30 19:54:00+00:00,29.055,29.055,29.055,29.055,147.0
57410,2021-12-30 19:59:00+00:00,29.120,29.120,29.120,29.120,109.0
57411,2021-12-30 20:08:00+00:00,29.100,29.100,29.100,29.100,431.0


In [6]:
df_date = complete_df[['date']].copy()
df_date['date'] = pd.to_datetime(df_date['date'])
df_date.head()

,date
57407,2021-12-30 19:47:00+00:00
57408,2021-12-30 19:52:00+00:00
57409,2021-12-30 19:54:00+00:00
57410,2021-12-30 19:59:00+00:00
57411,2021-12-30 20:08:00+00:00


In [7]:
df_value = complete_df.drop(columns=['date'])
df_value.head()

,open,high,low,close,volume
57407,29.070,29.070,29.060,29.060,530.0
57408,29.040,29.040,29.040,29.040,238.0
57409,29.055,29.055,29.055,29.055,147.0
57410,29.120,29.120,29.120,29.120,109.0
57411,29.100,29.100,29.100,29.100,431.0


In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df_value.values)
numpy_value = scaler.transform(df_value.values)
numpy_value.shape

(96, 5)

In [9]:
from utils.timefeatures import time_features

numpy_date = time_features(pd.to_datetime(df_date['date'].values), freq="h")
numpy_date = numpy_date.transpose(1, 0)
numpy_date.shape

(96, 4)

In [10]:
df_date.iloc[-10:]

,date
57493,2021-12-31 20:46:00+00:00
57494,2021-12-31 20:47:00+00:00
57495,2021-12-31 20:50:00+00:00
57496,2021-12-31 20:54:00+00:00
57497,2021-12-31 20:55:00+00:00
57498,2021-12-31 20:56:00+00:00
57499,2021-12-31 20:57:00+00:00
57500,2021-12-31 20:58:00+00:00
57501,2021-12-31 20:59:00+00:00
57502,2021-12-31 21:00:00+00:00


In [29]:
def generate_next_date(data: pd.DataFrame, freq: str = "min", step: int = 1, nb_sample: int = 10):
    last_time = pd.Timestamp(data['date'].values[-1])  # Convertir en Timestamp
    assert freq in ["min", "h", "d"], "freq must be in ['min', 'h', 'd']"
    
    if freq == "min":
        new_dates = [last_time + pd.Timedelta(minutes=step * i) for i in range(1, nb_sample + 1)]
    elif freq == "h":
        new_dates = [last_time + pd.Timedelta(hours=step * i) for i in range(1, nb_sample + 1)]
    elif freq == "d":
        new_dates = [last_time + pd.Timedelta(days=step * i) for i in range(1, nb_sample + 1)]
    
    new_df = pd.DataFrame(new_dates, columns=['date'])
    return new_df

In [30]:
next_date = generate_next_date(df_date, freq="min", step=1, nb_sample=10)
next_date

,date
0,2021-12-31 21:01:00
1,2021-12-31 21:02:00
2,2021-12-31 21:03:00
3,2021-12-31 21:04:00
4,2021-12-31 21:05:00
5,2021-12-31 21:06:00
6,2021-12-31 21:07:00
7,2021-12-31 21:08:00
8,2021-12-31 21:09:00
9,2021-12-31 21:10:00


In [59]:
import numpy as np

def build_batch(data:pd.DataFrame,size:list, freq:str="min", step:int=1):
    seq_len = size[0]
    label_len = size[1]
    pred_len = size[2]
    
    data = data.iloc[-seq_len:]
    data_date = data[['date']].copy()
    
    data_value = data.drop(columns=['date'])
    
    scaler = StandardScaler()
    scaler.fit(data_value.values)
    numpy_value_x = scaler.transform(data_value.values)
    
    numpy_date_x = time_features(pd.to_datetime(data_date['date'].values), freq=freq)
    numpy_date_x = numpy_date_x.transpose(1, 0)
    
    next_date = generate_next_date(data_date, freq=freq, step=step, nb_sample=pred_len)
    numpy_date_y = time_features(pd.to_datetime(next_date['date'].values), freq=freq)
    numpy_date_y = numpy_date_y.transpose(1, 0)
    
    numpy_date_y = np.concatenate([numpy_date_x[-label_len:], numpy_date_y], axis=0)
    
    return numpy_value_x, numpy_date_x, numpy_date_y

batch_x, batch_x_mark, batch_y_mark = build_batch(df, size, freq="h", step=1)
batch_x.shape, batch_x_mark.shape, batch_y_mark.shape

                            date
57407  2021-12-30 19:47:00+00:00
57408  2021-12-30 19:52:00+00:00
57409  2021-12-30 19:54:00+00:00
57410  2021-12-30 19:59:00+00:00
57411  2021-12-30 20:08:00+00:00


((96, 5), (96, 4), (72, 4))